In [73]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

class ESAgent:
    def __init__(self,weights = None, bias = None):
        """
            initialize agent with weight and bias

            The agent should carry a MLP network where it has 16 features input and 9 output.

            Hence, MLP has one input layer (16 weights), and input layer fully connects to one output layer (9 weights and 9 bias).

            In order to let ESAgent able to inherite network from its parent. The arguments should have default value for inilizate new population.
        Args:
            weights : weights of network or None
            bias : bias of network or None
        """
        if weights is not None:
            self.weights = weights
        else:
            self.weights = np.random.uniform(-1, 1, (16, 9))

        if bias is not None:
            self.bias = bias
        else:
            self.bias = np.random.uniform(-1, 1, 9)


    def make_a_move(self,board):
        """make a move respect to current board state. You would like to use feature_construct function to convert board state to input features

        Args:
            board: tic tak toe game state

        Returns:
            move (0-8): position of board

        """
        features = feature_construct(board)
        net = np.dot(features, self.weights) + self.bias
        output = 1 / (1 + np.exp(-net))
        output[board.flatten() != 0] = -np.inf
        move = np.argmax(output)
        return move
    
# dont change any code after this
class RuleBaseAgent:
    def __init__(self,id,rival_id,p_rnd=0.1):
        self.p_rnd = p_rnd
        self.move = -1
        self.id = id
        self.rival_id = rival_id
    
    def make_a_move(self,board):
        self.find_avaliable_position(board)
        if np.random.random() < self.p_rnd:
            self.random_move()
        elif self.make_win_move(board):
            pass
        elif self.make_block_move(board):
            pass
        elif self.make_two_open_move(board):
            pass
        else: 
            self.random_move()
        self.avaliable_moves = None
        return self.move
        
    def find_avaliable_position(self,board):
        self.avaliable_moves = [i for i in range(9) if board[i//3][i%3] == 0]

    def random_move(self):
        move = np.random.choice(self.avaliable_moves)
        # move = self.avaliable_moves[0]
        self.move = (move//3,move%3)

    def make_win_move(self,board):
        for i,row in enumerate(board):
            if row.sum() == 2 * self.id:
                for j,value in enumerate(row):
                    if value == 0:
                        self.move= (i,j)
                        return True
                    
        
        for j,col in enumerate(board.T):
            if col.sum() == 2 * self.id:
                for i,value in enumerate(col):
                    if value == 0:
                        self.move= (i,j) 
                        return True
                    
        if board.trace() == 2 * self.id:
            for i in range(3):
                if board[i][i] == 0:
                    self.move = (i,i)
                    return True
        
        if np.fliplr(board).trace() == 2 * self.id:
            for i in range(3):
                if board[i][2-i] == 0:
                    self.move = (i,2-i)
                    return True
        
        return False
    
    def make_block_move(self,board):
        for i,row in enumerate(board):
            if row.sum() == 2 * self.rival_id:
                for j,value in enumerate(row):
                    if value == 0:
                        self.move= (i,j)
                        return True
                    
        for j,col in enumerate(board.T):
            if col.sum() == 2 * self.rival_id:
                for i,value in enumerate(col):
                    if value == 0:
                        self.move= (i,j)
                        return True
    
        if board.trace() == 2 * self.rival_id:
            for i in range(3):
                if board[i][i] == 0:
                    self.move = (i,i)
                    return True
        
        if np.fliplr(board).trace() == 2 * self.rival_id:
            for i in range(3):
                if board[i][2-i] == 0:
                    self.move = (i,2-i)
                    return True
        
        return False
    
    def make_two_open_move(self,board):
        p = 0.5
        if board.trace() == self.id:
            for i in range(3):
                if board[i][i] == 0:
                    if p < np.random.random():
                        self.move = (i,i)
                        return True
                    else:
                        p = 1
        
        p = 0.5
        if np.fliplr(board).trace() == self.id:
            for i in range(3):
                if board[i][2-i] == 0:
                    if p < np.random.random():
                        self.move = (i,2-i)
                        return True
                    else:
                        p = 1
        
        p = 0.5
        for i,row in enumerate(board):
            if row.sum() == self.id:
                for j,value in enumerate(row):
                    if value == 0:
                        if p < np.random.random():
                            self.move= (i,j)
                            return True
                        else:
                            p = 1
                    
        
        p = 0.5
        for j,col in enumerate(board.T):
            if col.sum() == self.id:
                for i,value in enumerate(col):
                    if value == 0:
                        if p < np.random.random():
                            self.move= (i,j)
                            return True
                        else:
                            p = 1

        return False
    

In [74]:
import pytest
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt 

class TestAgent:
    def test_ESAgent_network(self):
        agent = ESAgent()

        assert agent.weights.shape == (16,9)
        assert agent.bias.shape == (9,)

    def test_ESAgent_network(self):
        agent = ESAgent()
        
        board = np.array([[0,1,1],
                          [1,-1,-1],
                          [-1,-1,1]
                          ])
        move = agent.make_a_move(board)
        assert move == 0

        board = np.array([[-1,1,1],
                          [1,0,-1],
                          [-1,-1,1]
                          ])
        move = agent.make_a_move(board)
        assert move == 4
        
        board = np.array([[-1,1,1],
                          [1,-1,-1],
                          [0,-1,1]
                          ])
        move = agent.make_a_move(board)
        assert move == 6

class TestModel:
    def test_model_play(self):
        em = Evolutionary_Model(max_pop=1000,parent_percent=0.3)
        best_agent = em.play_tic_tak_toe(max_epoch=100)

        num_trials = 1000
        wins = 0
        loss = 0
        draw = 0
        currTrial = 0

        for _ in range(num_trials):
            currTrial += 1
            env = TicTakToe(rng=0)
            while not env.terminate:
                action = best_agent.make_a_move(env.board)
                env.step(action)
            if env.winner == Player.PLAYER1:
                wins += 1
            elif env.winner == Player.PLAYER2:
                loss += 1
            else:
                draw += 1
            env.reset()

        print("wins:",wins/num_trials,"loss",loss/num_trials,"draw",draw/num_trials)

        assert True
        

In [75]:

import copy

import numpy as np

    
class TicTakToe():
    """
        tic take tow environment. you dont need to change any code here

    """
        
    def __init__(self,rng):
        """initilize opponent rule base agent

        Args:
            rng (_type_): it the number of chance opponents make random move
        """
        self.player2 = RuleBaseAgent(id=Player.PLAYER2,rival_id=Player.PLAYER1,p_rnd=rng)
        self.reset()

    def reset(self):
        """
            reset game state
        """
        self.board = np.zeros((3,3)).astype('int')
        self.prev_board = copy.deepcopy(self.board)
        self.curr_player = Player.PLAYER1
        self.round = 1
        self.winner = 0
        self.terminate = False

    def step(self,action):
        """environment take a action from player 1 (in this case will be your agent), and it will take this move and call opponent agent also move. 

        Args:
            action      : the position of board

        Returns:
            board       : the board state
            prev_board  : the prev board state (no useful in assignment 2)
            terminate   : if game terminated
            self.winner : winner of game if terminated, Player 1 -> player 1 win, Player 2 -> player 2 win, 0 -> draw
        """
        if self.terminate:
            return self.board,self.prev_board,self.terminate,self.winner

        if not isinstance(action,list) and not isinstance(action, np.ndarray) and not isinstance(action, tuple):
            x,y = (action//3,action%3)
        elif len(action) == 2:
            x,y = action
        else:
            print("invalid input")
            assert RuntimeError
        
        self.move(x,y)
        self.update_round()

        if not self.terminate:
            self.prev_board = copy.deepcopy(self.board)
            action = self.player2.make_a_move(self.board)
            x,y = action
            self.move(x,y)
            self.update_round()

        return self.board,self.prev_board,self.terminate,self.winner
    
    def switch_player(self):
        if self.curr_player == Player.PLAYER1:
            self.curr_player = Player.PLAYER2
        else:
            self.curr_player = Player.PLAYER1

    def move(self,x,y):
        if x < 0 or x > 2 or y < 0 or y > 2:
            print("out of boundary.")
            return False
        elif self.board[x][y] != 0:
            print("occupied.")
            return False
        else:
            self.board[x][y] = self.curr_player
            self.last_move = (x,y)

            return True

    def is_win(self):
        x,y = self.last_move
        rows_check = self.board[x].sum() == 3 * self.curr_player
        cols_check = self.board[:,y].sum() == 3 * self.curr_player

        left_diagonal_check = False
        if x == y:
            left_diagonal_check = np.trace(self.board) == 3 * self.curr_player

        right_diagonal_check = False
        if x + y == 2:
            right_diagonal_check = np.trace(np.fliplr(self.board)) == 3 * self.curr_player

        return rows_check or cols_check or left_diagonal_check or left_diagonal_check or right_diagonal_check
    
    def update_round(self):
        if self.is_win():
            self.winner = self.curr_player
            self.terminate = True
        
        self.round += 1
        if self.round > 9:
            self.terminate = True
        else:
            self.switch_player()

    def show(self):
        """
            print the board state
        """
        print(self.board)
        

In [76]:
import numpy as np
import pandas as pd
import copy
from matplotlib import pyplot as plt 

class Evolutionary_Model:
    def __init__(self,max_pop = 1000, parent_percent = 0.2):
        """initilize evolutionary model and first population

        Args:
            max_pop (int, optional): maximum population at each epoch. Defaults to 1000.
            parent_percent (float, optional): the percentage of parent after selection. Defaults to 0.2.
        
        """
        self.max_pop = max_pop
        self.parent_percent = parent_percent

        self.best_agent = None
        self.best_reward = -1000
        self.population = self.initial_population()

    def play_tic_tak_toe(self, max_epoch = 100):
        """play tic tak toe evolutionary and return the best agent you have

        you should follow the persudocode in slides but you allow to some changes as long as it return the best agent you have in the model.

        1. intilize population
        2. do
        3.  fitness         <- evaluation
        3.  parent          <- selection
        4.  new_population  <- evolution
        5. return best_agent

        Returns:
            best_agent (ESagent): _description_
        """

        for epoch in range(max_epoch):
            rewords = []
            for agent in self.population:
                reword = self.evaluation(TicTakToe(rng=0),agent,num_trials=100)
                rewords.append(reword)
                if reword > self.best_reward:
                    self.best_reward = reword
                    self.best_agent = agent
            parent = self.selection(rewords,self.population)
            self.population = self.evolution(parent)
        return self.best_agent
    
    def initial_population(self):
        """initilize first population

        Returns:
            population (ESAgent[]): Array of Agents
        """
        return [ESAgent() for _ in range(self.max_pop)]
    
    def evaluation(self,env,agent,num_trials):
        """
            evaluate the reward for each agent. feel free to have your own reward function.
        """
        total_reword = 0
        for _ in range(num_trials):
            env.reset()
            while not env.terminate:
                action = agent.make_a_move(env.board)
                env.step(action)
            if env.winner == Player.PLAYER1:
                total_reword += 1
            elif env.winner == Player.PLAYER2:
                total_reword -= 2
            else:
                total_reword += 0
        return total_reword/num_trials

    def selection(self,rewards,population):
        """
            select the best fit in the population. feel free to have your own selection.
            Make sure you select parent according to parent_percent
        """
        rewards = np.array(rewards)
        population = np.array(population)
        sorted_indices = np.argsort(rewards)
        rewards = rewards[sorted_indices]
        sorted_population = population[sorted_indices]
        parent_size = int(self.parent_percent * len(population))
        return sorted_population[:parent_size]


    def evolution(self,parents):
        """
            evolute new population from parents. 

            be careful about how to reinforce children. You don't want your children perform same as parents and even worser than parents.

            feel free to have your own evolution. In MLP case, you would like to add some noises to weights and bias.
        """
        children = []
        for _ in range(len(parents)):
            parent = np.random.choice(parents)
            child_weights = parent.weights + np.random.normal(0, 0.1, size=parent.weights.shape)
            child_bias = parent.bias + np.random.normal(0, 0.1, size=parent.bias.shape)
            child = ESAgent(weights=child_weights, bias=child_bias)
            children.append(child)
        return np.concatenate((parents, children))

In [77]:
from enum import IntEnum

class Player(IntEnum):
    PLAYER1 = 1
    PLAYER2 = -1

In [78]:
import numpy as np

def feature_construct(board):
    """convert board state to 16 features

    Args:
        board (_type_): board state

    Returns:
        features: 16 input features
    """
    row = board.sum(axis=0)
    col = board.sum(axis=1)

    a_row = row + 1
    b_row = row - 1
    a_col = col + 1
    b_col = col - 1

    dig = board.trace()
    flip_dig = np.fliplr(board).trace()

    a_dig = dig + 1
    a_flip_dig = flip_dig + 1
    b_dig = dig - 1
    b_flip_dig = flip_dig - 1

    features = np.concatenate([a_row,b_row,a_col,b_col,[a_dig,b_dig,a_flip_dig,b_flip_dig]])
    features = 1/(1 + np.exp(-features))

    return features

In [79]:
TestAgent.test_ESAgent_network(None)
TestModel.test_model_play(None)

wins: 0.611 loss 0.133 draw 0.256
